In [13]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import AutoTokenizer
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_recall_curve, auc
import random
import os
# Убиарем предупреждение о симулинках
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

In [14]:
# Если возможно, пытаемся запустить на видеокарте
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Загрузка данных
data_path = "data/labeled.csv"
df = pd.read_csv(data_path)


# Используем токенизатор Hugging Face
# Токенизатор Hugging Face — это инструмент из библиотеки Transformers, который преобразует текст в числовой формат
tokenizer = AutoTokenizer.from_pretrained("ai-forever/ruBert-base")

# Максимальная длина последовательности и размер батч
MAX_LEN = 128
BATCH_SIZE = 32

# кастомный датасет
class CustomDataSet(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        row = self.dataframe.iloc[index]
        comment = row["comment"]
        # Строка преобразует метку label из столбца toxic (0 / 1) в тензор PyTorch с типом данных float32
        # Это нужно чтобы метки можно было использовать в вычислениях модели
        label = torch.tensor(row["toxic"], dtype=torch.float32)

        # Токенизация текста
        encoding = self.tokenizer(
            # текст токенизации
            comment,
            # Дополняем последовательности до максимальной длины
            padding="max_length",
            # Обрезаем текст до максимальной длины
            truncation=True,
            # Максимальная длина текста
            max_length=self.max_len,
            # Возвращаем результат в виде PyTorch тензоров
            return_tensors="pt",
        )

        # извлекаем из результата токенизации два ключевых тензора:

        # индексы токенов текста.
        input_ids = encoding["input_ids"].squeeze(0)
        # маска внимания. указывает модели, какие токены учитывать
        attention_mask = encoding["attention_mask"].squeeze(0)

        return {
            # Возвращаем текст в виде токенов
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "label": label,
        }


# Создаем датасет
dataset = CustomDataSet(df, tokenizer, MAX_LEN)

# Разделяем на обучающую и тестовую выборки 80 / 20 используя random_split
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Используем даталоадеры
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# моедль GRU
class LabModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(LabModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_ids, attention_mask):
        embedded = self.embedding(input_ids)  # [batch_size, seq_len, embedding_dim]
        _, hidden = self.gru(embedded)  # hidden: [1, batch_size, hidden_dim]
        output = self.fc(hidden.squeeze(0))  # [batch_size, output_dim]
        return self.sigmoid(output)


# Параметры модели

# Количество уникальных токенов в словаре
VOCAB_SIZE = tokenizer.vocab_size
# Размер эмбеддингов (векторов слов)
EMBEDDING_DIM = 128
# Размер скрытого слоя GRU
HIDDEN_DIM = 64
# Размер выходного слоя (1 для бинарной классификации)
OUTPUT_DIM = 1

model = LabModel(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM).to(device)

# Оптимизатор отвечает за обновление параметров модели во время обучения.
optimizer = torch.optim.Adam(
    # список параметров модели
    model.parameters(),
    # скорость обучения learning rate
    lr=3e-4)
criterion = nn.BCELoss()

# обучение
def train_model(model, dataloader, optimizer, criterion):
    model.train()
    total_loss = 0

    for batch in dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        # Обнуляем градиенты
        optimizer.zero_grad()
        # Прогоняем данные через модель
        outputs = model(input_ids, attention_mask).squeeze(1)
        # Вычисляем ошибку
        loss = criterion(outputs, labels)
        # Вычисляем градиенты
        loss.backward()
        # Обновляем параметры модели
        optimizer.step()

        # Суммируем потери
        total_loss += loss.item()

    return total_loss / len(dataloader)


# валидация
def _evaluate_model(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0

    all_labels = []
    all_preds = []
    all_probs = []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)

            outputs = model(input_ids, attention_mask).squeeze(1)  # Предсказанные вероятности
            loss = criterion(outputs, labels)                     # Вычисляем ошибку
            total_loss += loss.item()

            # Сохраняем истинные метки и предсказания
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(outputs.cpu().numpy())
            # Классификация: вероятности > 0.5 -> класс 1
            all_preds.extend((outputs > 0.5).cpu().numpy())

    # Рассчитываем метрики
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    roc_auc = roc_auc_score(all_labels, all_probs)

    precision, recall, _ = precision_recall_curve(all_labels, all_probs)
    pr_auc = auc(recall, precision)
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1: {f1:.4f}")
    print(f"ROC-AUC: {roc_auc:.4f}")
    print(f"PR-AUC: {pr_auc:.4f}")

    return total_loss / len(dataloader), accuracy, f1, roc_auc, pr_auc


# обучение
EPOCHS = 5

for epoch in range(EPOCHS):
    train_loss = train_model(model, train_loader, optimizer, criterion)
    val_loss, val_accuracy, val_f1, _, _ = _evaluate_model(model, test_loader, criterion, device)

    print(f"Epoch {epoch + 1}/{EPOCHS}")
    print(f"Train Loss: {train_loss:.4f}")
    # print(f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, Val F1: {val_f1:.4f}")

Using device: cpu
Accuracy: 0.6490
F1: 0.0000
ROC-AUC: 0.5776
PR-AUC: 0.3979
Epoch 1/5
Train Loss: 0.6373
Accuracy: 0.6490
F1: 0.0000
ROC-AUC: 0.6187
PR-AUC: 0.4600
Epoch 2/5
Train Loss: 0.6314
Accuracy: 0.7825
F1: 0.6709
ROC-AUC: 0.8236
PR-AUC: 0.7350
Epoch 3/5
Train Loss: 0.5775
Accuracy: 0.8054
F1: 0.6857
ROC-AUC: 0.8624
PR-AUC: 0.7930
Epoch 4/5
Train Loss: 0.4189
Accuracy: 0.8293
F1: 0.7270
ROC-AUC: 0.8820
PR-AUC: 0.8204
Epoch 5/5
Train Loss: 0.3294


In [54]:
# Проверка работы на случайных данных
def predict_random_sample(model, dataset, amount = 5):
    model.eval()

    for i in range(amount):
        sample_idx = random.randint(0, len(dataset) - 1)
        sample = dataset[sample_idx]

        input_ids = sample["input_ids"].unsqueeze(0).to(device)
        attention_mask = sample["attention_mask"].unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(input_ids, attention_mask).item()

        prediction = "Токсичный" if output > 0.5 else "Не токсичный"
        print()
        print(f"Настоящий тип: {'Токсичный' if df.iloc[sample_idx]['toxic'] == 1.0 else 'Не токсичный'} | Предсказание: {prediction}")
        print(f"Комментарий: {df.iloc[sample_idx]['comment']}")
        print()


# Проверка работы на заданных вручную данных:
def predict_manual_input(_model, _tokenizer, text, device):

    # Ввод данных от пользователя
    columns = ["comment","toxic"]
    data = [[text,0]]

    # Создание DataFrame
    _df = pd.DataFrame(data, columns=columns)

    sample_idx = 0

    _dataset = CustomDataSet(_df, _tokenizer, MAX_LEN)
    sample = _dataset[sample_idx]

    input_ids = sample["input_ids"].unsqueeze(0).to(device)
    attention_mask = sample["attention_mask"].unsqueeze(0).to(device)

    with torch.no_grad():
        output = _model(input_ids, attention_mask).item()

    prediction = "Токсичный" if output > 0.5 else "Не токсичный"
    print()
    print(f"Предсказание: {prediction}")
    print(f"Комментарий: {_df.iloc[sample_idx]['comment']}")

def predict_programm(_model, _tokenizer, device):
    while True:
        q = str(input())
        if q in ["q", "quit", ""]:
            break
        else:
            predict_manual_input(_model, _tokenizer, q, device)

In [60]:
predict_random_sample(model, dataset, 2)


Настоящий тип: Не токсичный | Предсказание: Не токсичный
Комментарий: Говорят в администрации президента вызвали учителя из Китая....



Настоящий тип: Токсичный | Предсказание: Токсичный
Комментарий: Так высрал же тебе на лицо, а ты утерся и отреагировал, ну че ты как маленький.




In [61]:
predict_programm(model, tokenizer, device)


Предсказание: Не токсичный
Комментарий: Автор всё правильно сказал, я с ним согласен
